In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
from icesea2.experiments import gen_lhs_from_dict, norm_to_values, topography_to_icesheet_nc, gen_run_configs
from icesea2.margins import shp_series_to_icesheet
from icesea2.shearstress import make_shear_stress_map
from icesea2.models import IcesheetModel

In [3]:
shear_stress_map_name = "ely_victor_eurasia"
shear_stress_map_file = "/nfs/annie/cm15ogp/phd/data/shear_stress/eurasia/regions.shp"

In [4]:
dx = 5000
dy = 5000
x_min = -1265453.0
x_max = 4159547.0
y_min = -4722734.8
y_max = 1352265.2
grid_x = np.arange(x_min, x_max+dx, dx)
grid_y = np.arange(y_min, y_max+dy, dy)

grid_crs = '+ellps=WGS84 +proj=laea +lon_0=0.0 +lat_0=90 +x_0=0.0 +y_0=0.0 +no_defs'

In [5]:
times = [-140000]

In [12]:
shear_stress_parameters_var_ranges = {
    "g_pgm_ice_streams_ice_stream": [5000,20000],
    "g_sediment_marine_sediment": [10000,30000],
    "g_sediment_onshore_sediment": [30000,100000],
    "g_sediment_bedrock": [100000,150000],
    "p_ice_stream_interior_dist": [0,1000],
    "p_cold_ice_shear_stress": [120000,200000],
    "p_cold_ice_interior_dist": [1000,300],
    "p_hybrid_ice_stream_shear_stress": [5000,100000],
    "p_hybrid_ice_stream_dist": [0,600],
}
shear_stress_parameters_const = {
    "m_base_map_name": shear_stress_map_name,
    "m_base_map_file": shear_stress_map_file,
    "m_gauss_blur": 1,
    "m_base_value": 5000,
    "m_combine_pattern": "((g_sediment | (g_pgm_ice_streams * p_ice_stream)) | p_hybrid_ice_stream) | p_cold_ice",
    "g_layers": ["sediment", "pgm_ice_streams"],
    "p_processes": ["ice_stream", "cold_ice", "hybrid_ice_stream"],
}

In [13]:
lhs_norm_path = "batchelor/lhs_norm.csv"
lhs_values_path = "batchelor/lhs_values.csv"

if Path(lhs_norm_path).exists():
    lhs = np.genfromtxt(lhs_norm_path, delimiter=",", skip_header=1)
    lhs_values = np.genfromtxt(lhs_values_path, delimiter=",", skip_header=1)
else:
    lhs = gen_lhs_from_dict(
        parameter_dict=shear_stress_parameters_var_ranges,
        n_samples=200,
        save=lhs_norm_path
    )

    lhs_values = norm_to_values(
        lhs_norm_values=lhs,
        parameter_ranges=list(shear_stress_parameters_var_ranges.values()),
        parameter_names=list(shear_stress_parameters_var_ranges.keys()),
        save=lhs_values_path
    )
exp_vars = pd.DataFrame(lhs_values, columns=list(shear_stress_parameters_var_ranges.keys()))

In [8]:
! ls /nfs/annie/cm15ogp/phd/data/ice_margins/batchelor_quickfix/-140000

margin.cpg  margin.dbf	margin.prj  margin.shp	margin.shx


In [9]:
margins_name = "batchelor_mis6"
margins_dir = "/nfs/annie/cm15ogp/phd/data/ice_margins/batchelor_quickfix"
run_dir = "/nfs/annie/cm15ogp/phd/projects/2020-12-01_icesheet_calibration/experiments/shearstress_lhs_process/batchelor/"



In [21]:
t_var_topo = False
e_var_topo = False
e_var_ss = True
t_var_ss = True
e_var_margins = False

In [14]:
for time in tqdm(times):
    margin_filename = str(Path(margins_dir) / str(time) / "margin.shp")
    shear_stress_parameters_margins = {
        "p_margin_name": margins_name,
        "p_margin_time": time,
        "p_margin_file": margin_filename,
    }
    for exp_name, values in tqdm(exp_vars.iterrows(), total=len(exp_vars)):
        inputs_dir_shear_stress = (
            Path(run_dir) / 
            'inputs' / 
            'shearstress' / 
            str(time) / 
            str(exp_name)
        )
        inputs_dir_shear_stress.mkdir(exist_ok=True, parents=True)
        shear_stress_parameters_var = values.to_dict()
        shear_stress_parameters = {
            **shear_stress_parameters_const, 
            **shear_stress_parameters_var, 
            **shear_stress_parameters_margins,
        }

        make_shear_stress_map(
            grid_x = grid_x, 
            grid_y = grid_y, 
            grid_crs = grid_crs, 
            parameters = shear_stress_parameters,
            save=str(inputs_dir_shear_stress / "shearstress.nc")
        )


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [15]:
inputs_dir_margins = (
    Path(run_dir) / 
    'inputs' / 
    'margins'
)
inputs_dir_margins.mkdir(exist_ok=True, parents=True)

shp_series_to_icesheet(
    margins_dir, 
    grid_crs, 
    str(inputs_dir_margins), 
    times=times
)

In [16]:
topography_name = "rtopo2_laea_5km_eurasia"
topography_nc = "/nfs/annie/cm15ogp/phd/data/topography/rtopo2/rtopo2_laea_5km_eurasia.nc"


In [17]:
# topography
inputs_dir_topography = (Path(run_dir) / "inputs/topography")
inputs_dir_topography.mkdir(exist_ok=True, parents=True)
topography_to_icesheet_nc(topography_nc=topography_nc, output_dir=inputs_dir_topography)

In [22]:
run_configs = []
for member, config in exp_vars.iterrows():
    config = config.to_dict()
    run_config = gen_run_configs(
        run_dir = str(run_dir),
        times = times,
        member_name = str(member),
        t_var_topo=t_var_topo,
        e_var_topo=e_var_topo,
        t_var_ss=t_var_ss,
        e_var_ss=e_var_ss,
        e_var_margins=e_var_margins
    )
    run_configs = run_configs + run_config

In [23]:
run_configs[0]

(['/nfs/annie/cm15ogp/phd/projects/2020-12-01_icesheet_calibration/experiments/shearstress_lhs_process/batchelor/inputs/margins/-140000.1',
  '/nfs/annie/cm15ogp/phd/projects/2020-12-01_icesheet_calibration/experiments/shearstress_lhs_process/batchelor/inputs/margins/-140000.2'],
 '/nfs/annie/cm15ogp/phd/projects/2020-12-01_icesheet_calibration/experiments/shearstress_lhs_process/batchelor/inputs/shearstress/-140000/0/shearstress.nc',
 '/nfs/annie/cm15ogp/phd/projects/2020-12-01_icesheet_calibration/experiments/shearstress_lhs_process/batchelor/inputs/topography/topography.nc',
 '/nfs/annie/cm15ogp/phd/projects/2020-12-01_icesheet_calibration/experiments/shearstress_lhs_process/batchelor/outputs_raw/0/-140000/')

In [25]:
ism = IcesheetModel("/nfs/annie/cm15ogp/code/icesheet/binaries/icesheet")
ism.run_pool(run_configs, cores=24)